# Einstein Toolkit Output with NRPy's ETLegacy Infrastructure

## Author: Zach Etienne

## This notebook walks through how NRPy's ETLegacy infrastructure can generate a complete Einstein Toolkit thorn: C glue code, `*.ccl` files, and `make.code.defn`. Step by step, we will build a tiny evolution thorn and inspect the generated files.

The focus here is on using the ETLegacy helper modules by example. We do not look inside those modules; instead we exercise them to see the code they generate.

### Required reading if you are unfamiliar with programming or the Einstein Toolkit. Otherwise, skim for reference and jump into the examples.
+ **[Python Tutorial](https://docs.python.org/3/tutorial/index.html)**
+ **[Einstein Toolkit Documentation](https://einsteintoolkit.org/documentation.html)**

### NRPy Source Code for the infrastructure used here:
* [ETLegacy boundary conditions](../edit/nrpy/infrastructures/ETLegacy/boundary_conditions.py)
* [ETLegacy interface.ccl builder](../edit/nrpy/infrastructures/ETLegacy/interface_ccl.py)
* [ETLegacy param.ccl builder](../edit/nrpy/infrastructures/ETLegacy/param_ccl.py)
* [ETLegacy schedule.ccl builder](../edit/nrpy/infrastructures/ETLegacy/schedule_ccl.py)
* [ETLegacy simple loop helper](../edit/nrpy/infrastructures/ETLegacy/simple_loop.py)
* [ETLegacy zero RHS helper](../edit/nrpy/infrastructures/ETLegacy/zero_rhss.py)
* [ETLegacy MoL registration helper](../edit/nrpy/infrastructures/ETLegacy/MoL_registration.py)
* [ETLegacy symmetry registration helper](../edit/nrpy/infrastructures/ETLegacy/Symmetry_registration.py)
* [ETLegacy code parameter reader](../edit/nrpy/infrastructures/ETLegacy/CodeParameters.py)
* [ETLegacy make.code.defn helper](../edit/nrpy/infrastructures/ETLegacy/make_code_defn.py)

# Table of Contents

The module is organized as follows:

1. [Step 1](#Step-1:-Initialize-core-NRPy/ETLegacy-modules-and-project-directory): Initialize core NRPy/ETLegacy modules and project directory
1. [Step 2](#Step-2:-Declare-grid-functions-and-code-parameters-for-a-toy-thorn): Declare grid functions and code parameters for a toy thorn
1. [Step 3](#Step-3:-Generate-MoL-and-RHS-helper-C-functions): Generate MoL and RHS helper C functions
1. [Step 4](#Step-4:-Register-symmetries-and-boundary-conditions): Register symmetries and boundary conditions
1. [Step 5](#Step-5:-Reading-Einstein-Toolkit-parameters-in-generated-C-code): Reading Einstein Toolkit parameters in generated C code
1. [Step 6](#Step-6:-Building-interface.ccl-for-an-evolution-thorn): Building interface.ccl for an evolution thorn
1. [Step 7](#Step-7:-Building-param.ccl-from-registered-code-parameters): Building param.ccl from registered code parameters
1. [Step 8](#Step-8:-Building-schedule.ccl-from-registered-C-functions): Building schedule.ccl from registered C functions
1. [Step 9](#Step-9:-Writing-C-files-and-make.code.defn): Writing C files and make.code.defn

# Step 1: Initialize core NRPy/ETLegacy modules and project directory
### \[Back to [top](#Table-of-Contents)\]

We start by importing the core NRPy modules needed to drive the ETLegacy infrastructure and by defining a small project directory and thorn name.

Throughout this notebook we:

* Use ETLegacy helpers to register C functions for our thorn.
* Ask the helpers to emit C code and `*.ccl` files into a simple project tree.
* Inspect the generated files directly from Python.

The key global setting is the NRPy infrastructure parameter. Setting it to `"ETLegacy"` ensures that grid functions and helper code are tailored for the Einstein Toolkit legacy Cactus infrastructure and that `gri.register_gridfunctions()` creates `ETLegacyGridFunction` instances.

In [1]:
# Step 1: Initialize core NRPy/ETLegacy modules and project directory

from pathlib import Path

import nrpy.c_function as cfc
import nrpy.grid as gri
import nrpy.params as par

import nrpy.infrastructures.ETLegacy.ETLegacy_include_header as etl_includes
import nrpy.infrastructures.ETLegacy.simple_loop as etl_loop
import nrpy.infrastructures.ETLegacy.MoL_registration as etl_mol
import nrpy.infrastructures.ETLegacy.zero_rhss as etl_zero
import nrpy.infrastructures.ETLegacy.Symmetry_registration as etl_sym
import nrpy.infrastructures.ETLegacy.boundary_conditions as etl_bc
import nrpy.infrastructures.ETLegacy.interface_ccl as etl_interface
import nrpy.infrastructures.ETLegacy.param_ccl as etl_param
import nrpy.infrastructures.ETLegacy.schedule_ccl as etl_schedule
import nrpy.infrastructures.ETLegacy.CodeParameters as etl_CP
import nrpy.infrastructures.ETLegacy.make_code_defn as etl_make

# Target the ETLegacy infrastructure so that gridfunction helpers and
# boundary-condition helpers behave as expected for Cactus.
par.set_parval_from_str("Infrastructure", "ETLegacy")

# Define a small Einstein Toolkit project and thorn name.
project_dir = "NRPy_ET_project"
thorn_name = "NRPyToyBSSN"

Path(project_dir).mkdir(exist_ok=True)
print(f"Project directory: {Path(project_dir).resolve()}")

# Inspect the standard ETLegacy include list:
print("Standard ETLegacy includes:")
for inc in etl_includes.define_standard_includes():
    print("  ", inc)

Project directory: /home/zetienne/virt/tutorial/1-intro/NRPy_ET_project
Standard ETLegacy includes:
   math.h
   cctk.h
   cctk_Arguments.h
   cctk_Parameters.h


# Step 2: Declare grid functions and code parameters for a toy thorn
### \[Back to [top](#Table-of-Contents)\]

The ETLegacy helpers rely on NRPy's global registries:

* `gri.glb_gridfcs_dict` stores metadata for each registered grid function.
* `par.glb_code_params_dict` stores NRPy code parameters, which become Cactus parameters in `param.ccl`.

Instead of mocking grid functions, we use NRPy's own registration routines from `grid.py`:

* `gri.register_gridfunctions()` to create scalar gridfunctions.
* The infrastructure flag `"ETLegacy"` ensures that these become `ETLegacyGridFunction` objects.

This is essential for features such as NewRad boundary conditions, which explicitly check that each gridfunction is an instance of `gri.ETLegacyGridFunction` and expect metadata like `f_infinity` and `wavespeed`.

In a full NRPy workflow these registries are filled as part of setting up physics and equations. Here we:

* Use `gri.register_gridfunctions()` to create a small set of evolved and auxiliary gridfunctions.
* Provide simple NewRad metadata for evolved gridfunctions so that NewRad boundary-condition helpers can generate complete `NewRad_Apply` calls.
* Register a few code parameters, including a thorn-local `fd_order`, which controls ghost-zone sizes in some ETLegacy helpers.

In a production Einstein Toolkit run you would typically also have the standard `finite_difference::fd_order` parameter. Here we define a thorn-local `fd_order` that the ETLegacy helper modules expect; this is conceptually similar but lives in our toy thorn's namespace.

In [2]:
# Step 2: Declare toy gridfunctions and code parameters using NRPy's API
import sympy as sp

# Start from a clean slate for this notebook.
gri.glb_gridfcs_dict.clear()

# Register a couple of evolved variables (toy BSSN-like variables).
gri.register_gridfunctions(["phi", "trK"], group="EVOL")

# Register some auxiliary variables:
#   alpha:      scalar lapse
#   betaU0,1,2: components of the shift vector
gri.register_gridfunctions(["alpha"], group="EVOL", f_infinity=1.0, wavespeed=sp.sqrt(2))
gri.register_gridfunctions_for_single_rank1("betaU", group="EVOL")

# For NewRad boundary conditions, ETLegacy expects evolved gridfunctions
# to carry metadata describing the asymptotic value f_infinity and the
# characteristic wavespeed. In realistic thorns these would be physics-
# motivated expressions or parameters; here we choose simple constants.
for gfname, gf in gri.glb_gridfcs_dict.items():
    if getattr(gf, "group", None) == "EVOL":
        if not hasattr(gf, "f_infinity"):
            # Asymptotic value of the field at spatial infinity.
            gf.f_infinity = "0.0"
        if not hasattr(gf, "wavespeed"):
            # Characteristic speed of the outgoing mode at the boundary.
            gf.wavespeed = "1.0"

print("Toy gridfunctions registered in gri.glb_gridfcs_dict:")
for name, gf in sorted(gri.glb_gridfcs_dict.items()):
    info_pieces = [f"group={getattr(gf, 'group', 'unknown')}"]
    if hasattr(gf, "dimension"):
        info_pieces.append(f"dimension={gf.dimension}")
    if hasattr(gf, "f_infinity") and hasattr(gf, "wavespeed"):
        info_pieces.append(f"NewRad(f_infinity={gf.f_infinity}, wavespeed={gf.wavespeed})")
    print(f"  {name}: " + ", ".join(info_pieces))

# Register code parameters for this toy thorn.
# fd_order is used by ETLegacy helpers to compute ghost-zone sizes for AUX and boundary loops.
if "fd_order" not in par.glb_code_params_dict:
    par.register_CodeParameters(
        cparam_type="CCTK_INT",
        module=thorn_name,
        names=["fd_order"],
        defaultvalues=4,
    )

# Also register a couple of toy physical parameters that we will
# later read from generated C code using read_CodeParameters().
if "amplitude" not in par.glb_code_params_dict:
    par.register_CodeParameters(
        cparam_type="CCTK_REAL",
        module=thorn_name,
        names=["amplitude"],
        defaultvalues=1.0,
    )
if "sigma" not in par.glb_code_params_dict:
    par.register_CodeParameters(
        cparam_type="CCTK_REAL",
        module=thorn_name,
        names=["sigma"],
        defaultvalues=2.0,
    )

print("\nRegistered code parameters (subset):")
for name in ["fd_order", "amplitude", "sigma"]:
    CP = par.glb_code_params_dict.get(name, None)
    if CP is not None:
        print(f"  {name}: type={CP.cparam_type}, default={CP.defaultvalue}")

Toy gridfunctions registered in gri.glb_gridfcs_dict:
  alpha: group=EVOL, dimension=3, NewRad(f_infinity=1.0, wavespeed=sqrt(2))
  betaU0: group=EVOL, dimension=3, NewRad(f_infinity=0.0, wavespeed=1.0)
  betaU1: group=EVOL, dimension=3, NewRad(f_infinity=0.0, wavespeed=1.0)
  betaU2: group=EVOL, dimension=3, NewRad(f_infinity=0.0, wavespeed=1.0)
  phi: group=EVOL, dimension=3, NewRad(f_infinity=0.0, wavespeed=1.0)
  trK: group=EVOL, dimension=3, NewRad(f_infinity=0.0, wavespeed=1.0)

Registered code parameters (subset):
  fd_order: type=CCTK_INT, default=4
  amplitude: type=CCTK_REAL, default=1.0
  sigma: type=CCTK_REAL, default=2.0


# Step 3: Generate MoL and RHS helper C functions
### \[Back to [top](#Table-of-Contents)\]

The Einstein Toolkit uses the Method of Lines (MoL) thorn to advance evolved variables in time. To plug into MoL, a thorn typically needs at least:

* A function that zeros all right-hand-side (RHS) gridfunctions, to avoid uninitialized values creating NaNs.
* A function that registers the evolved gridfunction groups with the MoL thorn.

The ETLegacy helpers automate both:

* `register_CFunction_zero_rhss(thorn_name)` populates a C function that loops over the grid and sets all `*_rhsGF` values to zero.
* `register_CFunction_MoL_registration(thorn_name)` populates a C function that calls the MoL registration routines for the appropriate gridfunction groups.

Beneath the hood, both helpers add entries to `cfc.CFunction_dict`. Here we call them and inspect the resulting C code.

In [3]:
# Step 3: Generate ET MoL helpers and inspect the C code

# Start from a clean CFunction registry for this notebook.
cfc.CFunction_dict.clear()

# Register a C function that zeros all RHS gridfunctions.
etl_zero.register_CFunction_zero_rhss(thorn_name=thorn_name)

# Register a C function that connects our gridfunction groups to MoL.
etl_mol.register_CFunction_MoL_registration(thorn_name=thorn_name)

print("C functions registered so far:")
for name, cfunc in sorted(cfc.CFunction_dict.items()):
    print(f"  {name}: returns {cfunc.cfunc_type}")

# Inspect the zero-RHS helper; it should contain a triple loop
# over the grid and assignments of the form (schematically) RHS = 0.0.
zero_name = f"{thorn_name}_zero_rhss"
print(f"\nFull C function for {zero_name}:\n")
print(cfc.CFunction_dict[zero_name].full_function)

C functions registered so far:
  NRPyToyBSSN_MoL_registration: returns void
  NRPyToyBSSN_zero_rhss: returns void

Full C function for NRPyToyBSSN_zero_rhss:

#include "cctk.h"
#include "cctk_Arguments.h"
#include "cctk_Parameters.h"

/**
 * Zero RHSs for NRPy-generated thorn NRPyToyBSSN
 */
void NRPyToyBSSN_zero_rhss(CCTK_ARGUMENTS) {
  DECLARE_CCTK_ARGUMENTS_NRPyToyBSSN_zero_rhss;
  DECLARE_CCTK_PARAMETERS;
#pragma omp parallel for
  for (int i2 = 0; i2 < cctk_lsh[2]; i2++) {
    for (int i1 = 0; i1 < cctk_lsh[1]; i1++) {
      for (int i0 = 0; i0 < cctk_lsh[0]; i0++) {
        phi_rhsGF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2)] = 0.0;
        trK_rhsGF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2)] = 0.0;
        alpha_rhsGF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2)] = 0.0;
        betaU0_rhsGF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2)] = 0.0;
        betaU1_rhsGF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2)] = 0.0;
        betaU2_rhsGF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2)] = 0.0;
      } // END LOOP: for (int i0 = 0; i0 < c

The zero-RHS function above uses a helper from `simple_loop.py` to generate clean nested `for` loops over the grid. We can use `simple_loop.simple_loop` directly whenever we want to embed a body of C code inside a standard Einstein Toolkit loop nest.

By default the helper produces OpenMP-parallelized loops over all grid points, including ghost zones. Here is a small example where we pretend to update a toy RHS in place.

In [4]:
# Demonstrate the simple_loop helper directly
from nrpy.helpers.generic import clang_format
loop_body = "rhs_phiGF = 0.0;  // toy update for demonstration"

loop_code = clang_format(etl_loop.simple_loop(
    loop_body=loop_body,
    enable_simd=False,
    loop_region="all points",
    enable_OpenMP=True,
))

print("Example simple_loop output:\n")
print(loop_code)

Example simple_loop output:

#pragma omp parallel for
for (int i2 = 0; i2 < cctk_lsh[2]; i2++) {
  for (int i1 = 0; i1 < cctk_lsh[1]; i1++) {
    for (int i0 = 0; i0 < cctk_lsh[0]; i0++) {
      rhs_phiGF = 0.0; // toy update for demonstration
    } // END LOOP: for (int i0 = 0; i0 < cctk_lsh[0]; i0++)
  } // END LOOP: for (int i1 = 0; i1 < cctk_lsh[1]; i1++)
} // END LOOP: for (int i2 = 0; i2 < cctk_lsh[2]; i2++)



# Step 4: Register symmetries and boundary conditions
### \[Back to [top](#Table-of-Contents)\]

ETLegacy provides helpers for building C functions that:

* Register symmetry parities for each gridfunction with the `SymBase` infrastructure.
* Register boundary conditions for both the `Driver` and `Boundary` thorns.
* Optionally generate calls to the NewRad outer boundary-condition driver for evolved variables.

These helpers inspect `gri.glb_gridfcs_dict` to discover:

* Which gridfunctions belong to the `EVOL`, `AUXEVOL`, and `AUX` groups, and
* For symmetries, how the names of the gridfunctions encode their tensor parity.

The high-level helpers are:

* `register_CFunction_Symmetry_registration_oldCartGrid3D(thorn_name)`
* `register_CFunctions(thorn_name)`, which internally calls:
    * `register_CFunction_specify_Driver_BoundaryConditions(thorn_name)`
    * `register_CFunction_specify_evol_BoundaryConditions(thorn_name)`
    * `register_CFunction_specify_aux_BoundaryConditions(thorn_name)`
    * `register_CFunction_specify_NewRad_BoundaryConditions_parameters(thorn_name)`

Because we used `gri.register_gridfunctions()` in Step 2 and provided NewRad metadata on the evolved gridfunctions, the NewRad helper can now generate fully populated `NewRad_Apply` calls.

In [5]:
# Step 4: Register symmetry and boundary-condition C helpers

# Symmetry registration based on gridfunction names and groups:
etl_sym.register_CFunction_Symmetry_registration_oldCartGrid3D(thorn_name=thorn_name)

# Driver, Boundary, AUX, and NewRad boundary-condition registration:
etl_bc.register_CFunctions(thorn_name=thorn_name)

print("C functions after symmetry and boundary-condition registration:")
for name in sorted(cfc.CFunction_dict.keys()):
    print(" ", name)

# Show just the first few lines of the symmetry-registration function.
sym_name = f"{thorn_name}_Symmetry_registration_oldCartGrid3D"
sym_code = cfc.CFunction_dict[sym_name].full_function
print(f"\nSymmetry-registration C function ({sym_name}) [first 30 lines]:\n")
for line in sym_code.splitlines()[:30]:
    print(line)

# Inspect an excerpt of the Driver boundary-condition helper.
bc_name = f"{thorn_name}_specify_Driver_BoundaryConditions"
bc_code = cfc.CFunction_dict[bc_name].full_function
print(f"\nDriver boundary-condition C function ({bc_name}) [first 30 lines]:\n")
for line in bc_code.splitlines()[:30]:
    print(line)

# And an excerpt of the NewRad parameter helper to see how our f_infinity
# and wavespeed metadata are used.
newrad_name = f"{thorn_name}_specify_NewRad_BoundaryConditions_parameters"
newrad_code = cfc.CFunction_dict[newrad_name].full_function
print(f"\nNewRad boundary-condition C function ({newrad_name}) [first 30 lines]:\n")
for line in newrad_code.splitlines()[:30]:
    print(line)

C functions after symmetry and boundary-condition registration:
  NRPyToyBSSN_MoL_registration
  NRPyToyBSSN_Symmetry_registration_oldCartGrid3D
  NRPyToyBSSN_specify_Driver_BoundaryConditions
  NRPyToyBSSN_specify_NewRad_BoundaryConditions_parameters
  NRPyToyBSSN_specify_aux_BoundaryConditions
  NRPyToyBSSN_specify_evol_BoundaryConditions
  NRPyToyBSSN_zero_rhss

Symmetry-registration C function (NRPyToyBSSN_Symmetry_registration_oldCartGrid3D) [first 30 lines]:

#include "Symmetry.h"
#include "cctk.h"
#include "cctk_Arguments.h"
#include "cctk_Parameters.h"

/**
 * Register symmetries for NRPy-generated thorn NRPyToyBSSN
 */
void NRPyToyBSSN_Symmetry_registration_oldCartGrid3D(CCTK_ARGUMENTS) {
  DECLARE_CCTK_ARGUMENTS_NRPyToyBSSN_Symmetry_registration_oldCartGrid3D;
  DECLARE_CCTK_PARAMETERS;

  // Stores gridfunction parity across x=0, y=0, and z=0 planes, respectively
  int sym[3];

  // Next register parities for each gridfunction based on its name
  //    (to ensure this algori

# Step 5: Reading Einstein Toolkit parameters in generated C code
### \[Back to [top](#Table-of-Contents)\]

Many Einstein Toolkit routines need access to run-time parameters such as:

* Finite-difference order (`fd_order` for this toy thorn).
* Physical scales (`amplitude`, `sigma`).

In Cactus, such quantities are exposed as parameters in `param.ccl` and are retrieved at run time via `CCTK_ParameterGet`. The ETLegacy helper `read_CodeParameters()` generates C code snippets that:

* Call `CCTK_ParameterGet` for each requested parameter.
* Optionally set up SIMD-friendly copies of the same quantities.

Usage pattern:

```python
snippet = read_CodeParameters(
    list_of_tuples__thorn_CodeParameter=[(thorn_name, "fd_order"), (thorn_name, "amplitude")],
    enable_simd=True,
    declare_invdxxs=True,
)
```

The returned string `snippet` can be inserted directly into the body of an ET C function generated by NRPy. We now generate both scalar and SIMD versions for the parameters we registered earlier.

In [6]:
# Step 5: Demonstrate read_CodeParameters helper

# Scalar version: no SIMD, but we still ask it to declare the inverse grid spacings.
scalar_snippet = etl_CP.read_CodeParameters(
    list_of_tuples__thorn_CodeParameter=[
        (thorn_name, "fd_order"),
        (thorn_name, "amplitude"),
        (thorn_name, "sigma"),
    ],
    enable_simd=False,
    declare_invdxxs=True,
)

print("Scalar CodeParameters snippet:\n")
print(scalar_snippet)

# SIMD version: useful when the surrounding code is operating on SIMD vectors.
simd_snippet = etl_CP.read_CodeParameters(
    list_of_tuples__thorn_CodeParameter=[
        (thorn_name, "fd_order"),
        (thorn_name, "amplitude"),
        (thorn_name, "sigma"),
    ],
    enable_simd=True,
    declare_invdxxs=True,
)

print("SIMD CodeParameters snippet:\n")
print(simd_snippet)

Scalar CodeParameters snippet:

const CCTK_REAL *amplitudeptr = CCTK_ParameterGet("amplitude", "NRPyToyBSSN", NULL);  // NRPyToyBSSN::amplitude
const CCTK_REAL amplitude = *amplitudeptr;
const CCTK_INT fd_order = CCTK_ParameterGet("fd_order", "NRPyToyBSSN", NULL);  // NRPyToyBSSN::fd_order
const CCTK_REAL *sigmaptr = CCTK_ParameterGet("sigma", "NRPyToyBSSN", NULL);  // NRPyToyBSSN::sigma
const CCTK_REAL sigma = *sigmaptr;
const CCTK_REAL invdxx0 = 1.0/CCTK_DELTA_SPACE(0);
const CCTK_REAL invdxx1 = 1.0/CCTK_DELTA_SPACE(1);
const CCTK_REAL invdxx2 = 1.0/CCTK_DELTA_SPACE(2);

SIMD CodeParameters snippet:

const CCTK_REAL *restrict NOSIMDamplitude = CCTK_ParameterGet("amplitude", "NRPyToyBSSN", NULL);  // NRPyToyBSSN::amplitude
const REAL_SIMD_ARRAY amplitude = ConstSIMD(*NOSIMDamplitude);  // NRPyToyBSSN::amplitude
const CCTK_INT fd_order = CCTK_ParameterGet("fd_order", "NRPyToyBSSN", NULL);  // NRPyToyBSSN::fd_order
const CCTK_REAL *restrict NOSIMDsigma = CCTK_ParameterGet("sigma", "NRPy

# Step 6: Building interface.ccl for an evolution thorn
### \[Back to [top](#Table-of-Contents)\]

Every Einstein Toolkit thorn needs an `interface.ccl` file describing:

* What the thorn implements (`implements:`).
* Which other thorns it depends on (`inherits:`).
* Which functions it requires or uses from other thorns.
* Publicly visible gridfunction groups and their tags.

The ETLegacy helper `construct_interface_ccl()` takes:

* The project directory and thorn name.
* A string listing the thorns to inherit from.
* A block of `USES FUNCTION` or `REQUIRES FUNCTION` declarations.
* Flags indicating whether this thorn is an evolution thorn and whether NewRad support is required.

It then writes a complete `interface.ccl` file into the thorn directory. We now build a simple file for our toy thorn and inspect its contents.

In [7]:
# Step 6: Build interface.ccl for our toy evolution thorn

inherits = "Grid ADMBase CoordBase Time MoL"
USES_INCLUDEs = """
# Example: ask for a couple of functions we know we will need.
CCTK_INT FUNCTION CCTK_GroupIndex(CCTK_STRING IN group_name)
USES FUNCTION CCTK_GroupIndex
"""

etl_interface.construct_interface_ccl(
    project_dir=project_dir,
    thorn_name=thorn_name,
    inherits=inherits,
    USES_INCLUDEs=USES_INCLUDEs,
    is_evol_thorn=True,
    enable_NewRad=True,
)

interface_path = Path(project_dir) / thorn_name / "interface.ccl"
print(f"interface.ccl written to: {interface_path}\n")

# Show the full interface.ccl file.
print(interface_path.read_text())

Checking /home/zetienne/virt/tutorial/1-intro/NRPy_ET_project/NRPyToyBSSN/interface.ccl...[no changes]
interface.ccl written to: NRPy_ET_project/NRPyToyBSSN/interface.ccl


# This interface.ccl file was automatically generated by NRPy.
#   You are advised against modifying it directly; instead
#   modify the Python code that generates it.

# With "implements", we give our thorn its unique name.
implements: NRPyToyBSSN

# By "inheriting" other thorns, we tell the Toolkit that we
#   will rely on variables/function that exist within those
#   functions.
inherits: Grid ADMBase CoordBase Time MoL

# Needed functions and #include's:

# Example: ask for a couple of functions we know we will need.
CCTK_INT FUNCTION CCTK_GroupIndex(CCTK_STRING IN group_name)
USES FUNCTION CCTK_GroupIndex


# Needed Method of Lines function
CCTK_INT FUNCTION MoLRegisterEvolvedGroup(CCTK_INT IN EvolvedIndex, CCTK_INT IN RHSIndex)
REQUIRES FUNCTION MoLRegisterEvolvedGroup

# Needed Boundary Conditions function
CC

# Step 7: Building param.ccl from registered code parameters
### \[Back to [top](#Table-of-Contents)\]

The `param.ccl` file exposes run-time parameters to the Einstein Toolkit parameter database. ETLegacy can generate this file automatically by:

* Inspecting all registered C functions for the current thorn.
* Collecting any NRPy code parameters those functions mark as being used by this thorn.
* Writing the corresponding Cactus parameter declarations with sensible defaults.

The helper `construct_param_ccl()` takes:

* The project directory and thorn name.
* An optional header string describing any `shares:` or `extends:` relations.

It returns the list of parameter names written to the file and writes `param.ccl` into the thorn directory.

In [8]:
# Step 7: Build param.ccl based on the CFunctions we have registered

shares_extends_str = f"shares: {thorn_name}\n"

registered_params = etl_param.construct_param_ccl(
    project_dir=project_dir,
    thorn_name=thorn_name,
    shares_extends_str=shares_extends_str,
)

param_path = Path(project_dir) / thorn_name / "param.ccl"
print(f"param.ccl written to: {param_path}\n")

print("Parameters recorded in param.ccl:")
for name in registered_params:
    print("  ", name)

print("\nFull param.ccl file:\n")
print(param_path.read_text())

Checking /home/zetienne/virt/tutorial/1-intro/NRPy_ET_project/NRPyToyBSSN/param.ccl...[written]
param.ccl written to: NRPy_ET_project/NRPyToyBSSN/param.ccl

Parameters recorded in param.ccl:
   fd_order

Full param.ccl file:

# This param.ccl file was automatically generated by NRPy.
#   You are advised against modifying it directly; instead
#   modify the Python code that generates it.
shares: NRPyToyBSSN


restricted:
CCTK_INT fd_order "(see NRPy for parameter definition)"
{
 *:* :: "All values accepted. NRPy does not restrict the allowed ranges of parameters yet."
} 4




# Step 8: Building schedule.ccl from registered C functions
### \[Back to [top](#Table-of-Contents)\]

The Einstein Toolkit scheduling system controls when each thorn function is called during a simulation. ETLegacy helps build a `schedule.ccl` file by:

* Scanning all NRPy-registered C functions for the current thorn.
* Collecting their requested schedule bins and schedule entries.
* Adding any additional schedule entries you provide.

The central helper is `construct_schedule_ccl()`, which needs:

* The project directory and thorn name.
* A `STORAGE:` stanza describing which gridfunction groups should have storage allocated and at which phases.
* An optional list of extra schedule-bin entries.

The `STORAGE:` string is passed through as-is, so you can control storage at whatever level of detail you require.

In [9]:
# Step 8: Build schedule.ccl from the CFunctions we have registered

# A minimal STORAGE stanza for our toy evol and RHS variables.
STORAGE = """STORAGE: evol_variables(everywhere)
STORAGE: evol_variables_rhs(everywhere)
"""

etl_schedule.construct_schedule_ccl(
    project_dir=project_dir,
    thorn_name=thorn_name,
    STORAGE=STORAGE,
    extra_schedule_bins_entries=None,
)

schedule_path = Path(project_dir) / thorn_name / "schedule.ccl"
print(f"schedule.ccl written to: {schedule_path}\n")

print("Full schedule.ccl file:\n")
print(schedule_path.read_text())

Checking /home/zetienne/virt/tutorial/1-intro/NRPy_ET_project/NRPyToyBSSN/schedule.ccl...[no changes]
schedule.ccl written to: NRPy_ET_project/NRPyToyBSSN/schedule.ccl

Full schedule.ccl file:

# This schedule.ccl file was automatically generated by NRPy.
#   You are advised against modifying it directly; instead
#   modify the Python code that generates it.

##################################################
# Step 0: Allocate memory for gridfunctions, using the STORAGE: keyword.
STORAGE: evol_variables(everywhere)
STORAGE: evol_variables_rhs(everywhere)


##################################################
# Step 1: Schedule functions in the Driver_BoundarySelect scheduling bin.

schedule NRPyToyBSSN_specify_Driver_BoundaryConditions in Driver_BoundarySelect
{
  LANG: C
  OPTIONS: meta
} "Register boundary conditions in PreSync bin Driver_BoundarySelect."

##################################################
# Step 2: Schedule functions in the BASEGRID scheduling bin.

schedule NRPyToyB

# Step 9: Writing C files and make.code.defn
### \[Back to [top](#Table-of-Contents)\]

At this point we have:

* Registered a collection of C functions in `cfc.CFunction_dict` for our toy thorn.
* Built `interface.ccl`, `param.ccl`, and `schedule.ccl`.

The remaining tasks to create a compilable Einstein Toolkit thorn are:

1. Write each registered C function into its own `.c` file in the thorn's `src/` directory.
2. Generate a `make.code.defn` file listing all of those `.c` files so that Cactus can build them.

The helper `output_CFunctions_and_construct_make_code_defn()` automates both steps. It:

* Iterates over the `cfc.CFunction_dict` to find functions belonging to our thorn.
* Writes each function body to `thorn_name/src/<function_name>.c`.
* Builds a `make.code.defn` file listing the generated `.c` files.

We now run this helper and inspect the resulting build metadata.

In [10]:
# Step 9: Emit C sources and construct make.code.defn

etl_make.output_CFunctions_and_construct_make_code_defn(
    project_dir=project_dir,
    thorn_name=thorn_name,
)

src_path = Path(project_dir) / thorn_name / "src"
makecode_path = src_path / "make.code.defn"

print(f"Source directory: {src_path}")
print("Contents of src directory:")
for p in sorted(src_path.glob("*.c")):
    print("  ", p.name)

print("\nmake.code.defn:\n")
print(makecode_path.read_text())

# Optionally, peek at one of the generated C files, for example the MoL registration helper.
mol_c_path = src_path / f"{thorn_name}_MoL_registration.c"
if mol_c_path.exists():
    print(f"\nExcerpt from {mol_c_path.name}:\n")
    for line in mol_c_path.read_text().splitlines()[:40]:
        print(line)

Checking /home/zetienne/virt/tutorial/1-intro/NRPy_ET_project/NRPyToyBSSN/src/NRPyToyBSSN_MoL_registration.c...[no changes]
Checking /home/zetienne/virt/tutorial/1-intro/NRPy_ET_project/NRPyToyBSSN/src/NRPyToyBSSN_Symmetry_registration_oldCartGrid3D.c...[no changes]
Checking /home/zetienne/virt/tutorial/1-intro/NRPy_ET_project/NRPyToyBSSN/src/NRPyToyBSSN_specify_Driver_BoundaryConditions.c...[written]
Checking /home/zetienne/virt/tutorial/1-intro/NRPy_ET_project/NRPyToyBSSN/src/NRPyToyBSSN_specify_NewRad_BoundaryConditions_parameters.c...[written]
Checking /home/zetienne/virt/tutorial/1-intro/NRPy_ET_project/NRPyToyBSSN/src/NRPyToyBSSN_specify_aux_BoundaryConditions.c...[written]
Checking /home/zetienne/virt/tutorial/1-intro/NRPy_ET_project/NRPyToyBSSN/src/NRPyToyBSSN_specify_evol_BoundaryConditions.c...[written]
Checking /home/zetienne/virt/tutorial/1-intro/NRPy_ET_project/NRPyToyBSSN/src/NRPyToyBSSN_zero_rhss.c...[written]
Checking /home/zetienne/virt/tutorial/1-intro/NRPy_ET_project